# Lecture 21

#### Complex Numbers; Default Arguments; (Incomplete) Poker

# 1. Complex Numbers, and Overloading Arithmetic Operators

Let's create a class for complex numbers.  (Granted, Python already has a data type for complex numbers, but that shouldn't stop us from creating our own.)  

Recall that complex numbers are numbers of the form $a+bi$, where $a, b$ are real numbers, and $i$ is the square root of $-1$.  So $5 + 7i$ is a complex number, as is $-4.2 - 6.1i$, as is $18$ (since $18 = 18 + 0i$).

Complex numbers are added via the rule $(a+bi) + (c+di) = (a+c) + (b+d)i$, and they are multiplied by the rule $(a+bi)(c+di) = ac + adi + bci +bdi^2 = (ac - bd) + (ad + bc)i$.  So, before we get too far into it, let's quickly remider ourselves how these work:

$(4 + 2i) + (3 - i) = ?$

$(2 + 3i)(1 - 4i) = ?$

So, we'll design our class so that each complex number has two attributes: a real part and an imaginary part, both `float`s.  We'll implement the $+$ and $*$ operators, and we'll make our complex numbers printable.

In [3]:
# EXAMPLE 1a: A Complex class


class Compl:
    
    def __init__(self, a, b):
        self._re = a
        self._im = b

    def __str__(self):
        if self._im < 0:
            return str(self._re) + str(self._im) + "i"
        else:
            return str(self._re) + "+" + str(self._im) + "i"
    
    
    
    # Overload +
    def __add__(self,other):
        new_re = self._re + other._re
        new_im = self._im + other._im
        return Compl(new_re,new_im)
    
    # Overload *
    def __mul__(self,other):
        new_re = (self._re * other._re) - (self._im * other._im)
        new_im = (self._re * other._im) + (self._im * other._re)
        return Compl(new_re,new_im)

####################
def main():
    z1 = Compl(2,3)   # This should represent the value 2 + 3i
    z2 = Compl(2,-3)  # This should represent the value 2 - 3i
    z3 = Compl(4,0.5) # This hsould represent the value 4 + 0.5i

    print("z1, z2, z3:")
    print(z1, z2, z3)

    print("\nz1 + z2 (should be 4 + 0i):")
    print(z1 + z2)

    print("\nz1 + z3 (should be 6 + 3.5i):")
    print(z1 + z3)

    print("\nz1 * z2 (should be 13 + 0i):")
    print(z1 * z2)

    print("\nz1 * z3 (should be 6.5 + 13i):")
    print(z1 * z3)

    print("\nz2 + z1 * z3 (should be 8.5 + 10i):")
    print(z2 + z1 * z3) 
    
####################
main()

z1, z2, z3:
2+3i 2-3i 4+0.5i

z1 + z2 (should be 4 + 0i):
4+0i

z1 + z3 (should be 6 + 3.5i):
6+3.5i

z1 * z2 (should be 13 + 0i):
13+0i

z1 * z3 (should be 6.5 + 13i):
6.5+13.0i

z2 + z1 * z3 (should be 8.5 + 10i):
8.5+10.0i


<br><br><br><br><br><br><br><br><br><br>

# 2. Default Arguments

The following applies to all functions, whether or not they are class methods.  However, it seems to be particularly useful for constructors. 

When we've written constructors so far, they have tended to look like this:            

In [ ]:
def __init__(self, r, s):
    self._rank = r
    self._suit = s

and then, when we actually initialize an argument, there would be (in this case) *two* parameters, to set the rank and the suit (the first parameter, `self`, is implicit): so you would initialize an object in the form `x = Card("2", "Clubs")`.  

However, it is sometimes nice to have a "default" initialization option as well, if we want to create a playing card without immediately specifying what that card is.  We can do this by using by supplying *default arguments* to the constructor function, as follows.

In [4]:
# EXAMPLE 2a: Default arguments

class Card:
    """Represent a playing card.  Attributes: _rank and _suit"""
    
    # This constructor now has DEFAULT ARGUMENTS: if you create an object without specifying input values, then 
    # r will be assigned "None" and s will be assigned "None".
    # If you DO supply values, the r and s will take on those values, and ignore the defaults.
    def __init__(self, r = "Blank", s = "Blank"):
        self._rank = r
        self._suit = s
        
    def display(self):
        print(self._rank, "of", self._suit)
        
#########################       
c1 = Card() # No arguments here!  So the default values will be supplied
c2 = Card("2", "Spades") # Initialized with arguments, so the default values will be ignored

# These are technically allowed by Python, although caution is warranted.  Basic rule: default arguments should come
# as the last argument(s) (both when calling a function and when defining one).
c3 = Card("3") 
c4 = Card("Diamonds") # I don't think this one works the way we've planned it.
c1.display()
c2.display()
c3.display()
c4.display()

Blank of Blank
2 of Spades
3 of Blank
Diamonds of Blank


So, when defining any function, you can supply default argument for the parameters, which will be used if no values are supplied, and ignored if values are supplied.

In practice, it might be a good idea (both when writing functions and calling them) to either make all your arguments have default values, or none of them: Python has to be able to figure out *which* arguments you're omitting, so if you omit some but not others, you can run into complications.  There are rules that Python uses for this (basic one: put default arguments at the end).

<br><br><br><br><br><br><br><br><br><br>

# 3. Poker

I think it's time to show you the (incomplete) poker game.

This is going to involve a number of classes.  Obviously, `Card` and `Deck` are going to come into play.  Here are reasonably sophisticated versions of these two classes.

In [ ]:
# EXAMPLE 3a: Card and Deck Classes

import random

class Card:
    def __init__(self, r = "None", s = "None"):
        self._rank = r
        self._suit = s
        
    def __str__(self):
        return self._rank + " of " + self._suit

    def value(self):
        """A helper function: return 0 if the rank is 2,
        1 if the rank is 3, ... 11 if the rank is King, 12 if the 
        rank is Ace"""
        rank_order = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King", "Ace"]
        return rank_order.index(self._rank)
    
    def suit(self):
        return self._suit
    
    # These three methods will be helpful!
    # Among other things, we can use them to sort hands.
    def __lt__(self, other):        
        return self.value() < other.value()
    def __gt__(self, other):
        return self.value() > other.value()
    def __eq__(self, other):
        # WARNING: Tests equality of ranks, not equality of cards!
        return self.value() == other.value()

    
##########################################################    
        
    
class Deck:
    def __init__(self):
        all_ranks = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King", "Ace"]
        all_suits = ["Hearts", "Clubs", "Diamonds", "Spades"]
    
        self._cards_left = 52
        self._cards = []
        #Create a list of all cards
        for r in all_ranks:
            for s in all_suits:
                self._cards.append(Card(r,s))
        # The "Knuth Shuffle"
        for i in range(52):
            j = random.randrange(i,52)
            temp = self._cards[i]
            self._cards[i] = self._cards[j]
            self._cards[j] = temp
        
    def draw(self):
        if self._cards_left > 0:
            self._cards_left -= 1
            # .pop() removes the last card from the deck, AND returns it.
            return self._cards.pop()
        else:
            # This simply gives an error message when the program crashes.
            raise Exception("EMPTY DECK")

<br><br><br><br><br><br><br><br><br><br>



As for the rest of the game: I'm going to utilize two more classes: one for Hands, and one for Players.  These are, after all, the nouns that you'd probably include in a description of the game.  

*It is extremely helpful to program these classes in a manner that matches with how we think of the real world objects. So, before we sit down to program, we should ask ourselves some questions:*

What attributes will a `Player` have?  What attributes will a `Hand` have?

<br><br><br><br><br><br><br><br><br><br>


`Player`: will have a name, and a `Hand`.  At the time of creation, we'll require a player to have a name, and the `Hand` will start out as `None`.

`Hand`: will have a list of 5 `Card`s. At the time of creation, we'll require a `Deck` to draw from, and it will fill the list randomly.

Next, what things will we do with a `Player`, and/or what will a `Player` do? What things will we do with a `Hand`, and/or what will a `Hand` do?

<br><br><br><br><br><br><br><br><br><br>




`Player`: a `Player` will `receive()` a hand.  A player will also `trade()` specified cards.  At certain times throughout the game, we'll need to be able to `get_hand()` from the `Player` (i.e., get the `Player`'s hand, to compare or display it).

`Hand`: we will compare `Hand`s (using overloaded comparison operators).  Also, when a player decides to `trade()` cards, we will `update()` the hand (using the game deck).  We may want to `display()` a `Hand` also.


------------
Let's look at the `Hand` class first.  This is the most complex class.  Every `Hand` is drawn from a deck, so the initializer needs to have a particular `Deck` as an outside argument.  Furthermore, when you `update()` a hand, you need to know which card out of the 5 you are updating, and (again) which `Deck` you are drawing from.  I've also included a handy `display()` method.

<br><br><br><br><br><br><br><br><br><br>


And then come the comparisons.  **These are super incomplete!** They are meant to give a basic taste of how the comparisons work, not the whole story.  

The basic strategy is: assign a hand a "score", which is 0 - 12 based on the rank of the high card if there is no pair, or 13 if there is a pair, or 14 if there is a three of a kind.  Note: there are obviously other hands, and sometimes you need to compare, e.g., two different pairs to see which one is higher.  You can come up with richer score functions that allow you to compare any two hands.

In identifying whether or not there is a pair in a given hand, I use the `sorted()` function.  This takes a list of anything, and produces a new list with the elements ordered so that each element is `<` than the next one.  This works with cards *specifically because we overloaded the `<` and `>` operators!*.

In [1]:
# EXAMPLE 3b: Hand class

class Hand:

    def __init__(self, deck):
        self._cards = []
        for i in range(5):
            self._cards.append(deck.draw())

    def display(self):
        for card in self._cards:
            print(card)

    def update(self, i, deck):
        self._cards[i] = deck.draw()

    ###########################################################################
    #
    # The following is very preliminary. There are a lot of details that need
    # to be fixed here: what if both players have a pair?  What if both players
    # have equal pairs? What about the other hands?
    # 
    # A richer "scoring" scheme can sort this out.
    #
    def _is_pair(self):
        # This magic Python function takes advantage of the fact that
        # we defined < and > on Card. It produces a new list of cards, 
        # which is sorted by rank.
        s = sorted(self._cards)

        
        if s[0] == s[1] and s[1] < s[2] < s[3] < s[4]:
            return True
        if s[1] == s[2] and s[0] < s[2] < s[3] < s[4]:
            return True
        if s[2] == s[3] and s[0] < s[1] < s[2] < s[4]:
            return True
        if s[3] == s[4] and s[0] < s[1] < s[2] < s[3]:
            return True
        
        return False
        
    def _is_triple(self):
        s = sorted(self._cards)

        if s[0] == s[1] == s[2] and s[2] < s[3] < s[4]:
            return True
        if s[1] == s[2] == s[3] and s[0] < s[2] < s[4]:
            return True
        if s[2] == s[3] == s[4] and s[0] < s[1] < s[2]:
            return True
        
        return False
        
    def _hand_score(self):
        #
        # An (incomplete) scheme for comparing hands:
        # Three of a kind = 14 points
        # Pair = 13
        # Otherwise = Value of high card (0-12)
        #
        # This can be improved.
        #
        if self._is_triple():
            return 14
        elif self._is_pair():
            return 13
        else:        
            # High card
            s = sorted(self._cards)
            return s[4].value()

    #
    #
    ###########################################################################
    
    #
    #
    # FINALLY, THESE FUNCTIONS COMPARE HANDS
    #
    #
    def __lt__(self, other):
        return self._hand_score() < other._hand_score()

    def __gt__(self, other):        
        return self._hand_score() > other._hand_score()

    def __eq__(self, other):
        return self._hand_score() == other._hand_score()


<br><br><br><br><br><br><br><br><br><br>


The `Player` class is considerably simpler.  Each `Player` will start with a name ("Computer" by default), and no hand (designated by the value `None`).  There is a function that allows each `Player` to `receive_hand()`; this function assigns `self._hand` to be a new `Hand`, which means that this function need a `Deck` as outside argument to work properly.  A `Player` may also `trade()` cards from their `self._hand`: this function requires a list of cards to trade AND a `Deck` to draw from -- and this function will take advantage of the `update()` function.  Finally, `get_hand()` will simply return `self._hand`, which we can then compare or display.

In [ ]:
# EXAMPLE 3c: Player class

class Player:

    def __init__(self, name = "Computer"):
        self._name = name
        self._hand = None

    def __str__(self):
        return self._name
    
    def receive_hand(self, deck):
        self._hand = Hand(deck)

    def trade(self, trade_list, deck):
        for i in trade_list:
            self._hand.update(i, deck)    

    def get_hand(self):
        """Return the player's hand"""
        return self._hand    

Can we put this all together now?

<br><br><br><br><br><br><br><br><br><br>


In [ ]:
# EXAMPLE 3d: Poker Game

def main():
    # The basic objects.
    name = input("Enter your name: ")
    
    
    
    
    # Give hands
    
    
    
    # Player 1's hand
    print("\n" + name + ", here are your cards:\n")
    
    
    
    # Player 1 trades up
    trades = input("Which cards do you want to trade? ")
    trade_list = []
    for n in trades.split():
        trade_list.append(int(n) - 1)
    
    
    
    # Player 1's new hand
    print("\n" + name + ", here are your cards:\n")
    
    
    
    # Computer's reveal
    print("\nNow, here are my cards:\n")
        
    # Who wins?
    
    
    
########################################
main()
